In [247]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import IPython

# adding these modules to slow down crawl-rate
# prevents us from being banned
from time import time, sleep
from random import randint



product_name = []
product_image = []
product_short_description = []
product_description = []
brand_name = []
brand_description = []
product_href = []
product_detail_path = []

pages = [str(i) for i in range(45,46)]

for page in pages:
    response = requests.get('https://www.leafly.com/products/cannabis/flower?page=' + page)
    soup = BeautifulSoup(response.text, 'html.parser')
    product_containers = soup.find_all('a',{'class':'item'})
    
#   pause the loop between 8 - 15 seconds
#     sleep(randint(2,5))

    for container in product_containers:
        name = container.find('div', class_ = 'product-name').text
        product_name.append(name)

        image = container.find('img', attrs = {'class':'b-lazy'})['data-src']
        product_image.append(image)

        href = container['href']
        product_href.append(href)

        product_detail_url = 'https://www.leafly.com' + href
        product_detail_path.append(product_detail_url)

    for product_url in product_detail_path:
        response_detail = requests.get(product_url)
        product_detail_soup = BeautifulSoup(response_detail.text, 'html.parser')
        product_detail_sd = product_detail_soup.find('p', class_ = "product-short-description")
        product_short_description.append(product_detail_sd.text.strip())
        
        product_long_description = product_detail_soup.find('p', class_= "product-description")
        product_description.append(product_long_description.text)

        product_brand_name = product_detail_soup.find('p', class_ = 'brand-name').a.text
        brand_name.append(product_brand_name)
        
        product_brand_description = product_detail_soup.find('p', class_ = 'brand-description').text
        brand_description.append(product_brand_description)

product_detail_df = pd.DataFrame ({
    '1 Product Name' : product_name,
    '2 Product Image' : product_image,
    '3 Short Description' : product_short_description,
    '4 Product Description' : product_description,
    '5 Brand Name' : brand_name,
    '6 Brand Description' : brand_description,
})

In [248]:
# product_detail_df.to_csv('cp45.csv', index=False, encoding='utf-8')